## Tool Calling

Function calling lets a language model go beyond just generating text — it can decide when to use specific tools or functions based on what the user says. For example, if someone asks “what’s 3 times 5?”, the model can respond with a structured output like {"name": "multiply", "arguments": {"x": 3, "y": 5}}, which your program can then use to run the actual multiply function. This works by giving the model a clear prompt that explains what tools are available and how to respond in a specific format, usually JSON. With the right setup, even models that don’t have built-in tool support can still follow the instructions and act like smart assistants that trigger real code.

## Functions

In this tutorial, we’ll be working with functions and using function calling to make language models more useful and interactive. Function calling is especially powerful when the model needs to handle tasks it can’t do on its own — like checking the current weather or time, accessing private or proprietary data, or querying an external system like a SQL database. By setting up a system where the model can “ask” for a specific tool to be used, we can bridge the gap between the LLM and real-world functionality it wouldn’t otherwise have access to.

Lets take a look at some functions that an LLM would not naturally be able to get:

In [1]:
from datetime import datetime
import json

# Function to get the current time
def get_current_time():
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    return json.dumps({"current_time": current_time})

get_current_time()

'{"current_time": "2025-06-24 17:01:16"}'

In [2]:
from weather import get_weather
get_weather("La Jolla")

'{"city": "La Jolla", "temperature": 19.4, "wind_speed": 11.4, "condition": "clear sky \\u2600\\ufe0f"}'

In [3]:
import sqlite3

# Create an in-memory SQLite database and populate it with sample data
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Create a sample table and insert some data
cursor.execute('''
    CREATE TABLE employees (
        id INTEGER PRIMARY KEY,
        name TEXT,
        position TEXT,
        salary INTEGER
    )
''')

sample_data = [
    (1, 'Alice', 'Engineer', 75000),
    (2, 'Bob', 'Manager', 90000),
    (3, 'Charlie', 'Analyst', 60000),
    (4, 'Diana', 'HR', 50000)
]

cursor.executemany('INSERT INTO employees VALUES (?, ?, ?, ?)', sample_data)
conn.commit()

# Define the function to execute SQL queries
def execute_sql_query(query: str):
    try:
        cursor.execute(query)
        results = cursor.fetchall()
        return json.dumps({"results": results})
    except Exception as e:
        return json.dumps({"error": str(e)})

In [4]:
execute_sql_query("SELECT * FROM employees")

'{"results": [[1, "Alice", "Engineer", 75000], [2, "Bob", "Manager", 90000], [3, "Charlie", "Analyst", 60000], [4, "Diana", "HR", 50000]]}'

## Describe Function

Before an LLM can use a function, we need to describe that function in a way the model can understand. This is usually done using a JSON-like dictionary that includes:

- The function’s name

- A description of what it does

- Its parameters, including types and optional descriptions for each argument

You can write these descriptions manually, or even ask an LLM to help generate them — and then review or modify the result as needed.

In [5]:
weather_tool = {
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Fetches the current weather for a given city.",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "The name of the city."
                }
            },
            "required": ["city"]
        }
    }
}


time_tool = {
    "type": "function",
    "function": {
        "name": "get_current_time",
        "description": "Fetches the current time.",
        "parameters": {}
    }
}

sql_tool = {
    "type": "function",
    "function": {
        "name": "execute_sql_query",
        "description": "Execute an SQL query on database with table name 'employees' and return the results. The schema is id, name, position, salary",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "The SQL query to execute."
                }
            },
            "required": ["query"]
        }
    }
}

## Tool Calling Setup

In [6]:
TOOL_FUNCTIONS = {
    "get_weather": get_weather,
    "get_current_time": get_current_time,
    "execute_sql_query": execute_sql_query
}

In [7]:
import os
from ollama import Client

# Read the port from the file
with open(os.path.expanduser('~/.ollama_port')) as f:
    port = f.read().strip()

# Connect to 127.0.0.1:<port>
host = f"http://127.0.0.1:{port}"

client = Client(host=host)

In [8]:
# Get LLM
client.pull("qwen3:4b")

ProgressResponse(status='success', completed=None, total=None, digest=None)

System prompts allow the programmer to achieve greater control over the LLM's use of tools and higher-level context. 

They are not visible to the user but instead are a distinct section of the LLM's context window. System prompts often include phrases like "Use tools only when necessary." or give the LLM useful meta-context such as "You are a helpful assistant."

Here you can experiment with the system prompt and understand how it affects the later queries.

In [9]:
system_prompt = "Use tools only when explicitly necessary."

def chat(prompt):
    response = client.chat(
        model='qwen3:4b',
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
        tools=[weather_tool, time_tool, sql_tool],
        options={
            'temperature': 0
        }
    )

    if "tool_calls" in response["message"]:
        for tool_call in response["message"]["tool_calls"]:
            tool_name = tool_call["function"]["name"]
            args = tool_call["function"].get("arguments", {})
            print(f"Tool: {tool_name}")

            # Ensure the tool exists
            if tool_name not in TOOL_FUNCTIONS:
                print(f"Tool '{tool_name}' is not implemented.")
                continue

            try:
                # Call the actual tool function
                tool_output = json.loads(TOOL_FUNCTIONS[tool_name](**args))
            except Exception as e:
                tool_output = {"error": str(e)}

            if "error" in tool_output:
                final_response = tool_output["error"]
            else:
                followup = client.chat(
                    # model='llama3.1',
                    model='qwen3:4b',
                    messages=[
                        {"role": "system", "content": f"You are a helpful assistant. Turn tool outputs into natural conversational replies. If weather tool was used, please convert celsius to fahrenheit."},
                        {"role": "user", "content": f"Here is the output of the tool '{tool_name}': {tool_output}, with the prompt: {prompt}. Reply as an answerer."}
                    ]
                )
                final_response = followup["message"]["content"]

            print(final_response)
    else:
        print(response["message"]["content"])


## Using the Tool-calling LLM

- Ask some time related questions (May take some time to complete)

In [10]:
chat("What time is it, and how long will it take to reach the next hour.")

Tool: get_current_time
<think>
Okay, let's see. The user asked, "What time is it, and how long will it take to reach the next hour?" The tool response is the current time as '2025-06-24 17:01:43'. 

First, I need to parse that time. The date is 2025-06-24, which is June 24th, 2025. The time is 17:01:43, which is 5:01:43 PM in 12-hour format. 

The first part of the question is asking for the current time. So I should state the time in a natural way. Maybe say "It is currently 5:01:43 PM on June 24th, 2025."

Next, the user wants to know how long until the next hour. The current time is 17:01:43, so the next hour would be 18:00:00. Let's calculate the difference. From 17:01:43 to 18:00:00 is 48 minutes and 17 seconds. Wait, let me check that again. 

From 17:01:43 to 17:00:00 is 1 minute and 43 seconds before, but since we're going to the next hour, it's from 17:01:43 to 18:00:00. So subtract 17:00:00 from 18:00:00, which is 1 hour. Then subtract the current time from 17:00:00. Wait, ma

- Ask what the weather is like in whichever city you want to ask about.

In [11]:
chat("What is the current weather in San Diego, and will it be good for surfing?")

Tool: get_weather
<think>
Okay, the user is asking about the current weather in San Diego and if it's good for surfing. Let me check the tool response. The temperature is 21.4°C, which is around 70.5°F. The wind speed is 11.3 m/s, that's pretty strong. The condition is clear sky with a sun icon. 

First, I need to present the current weather details clearly. Convert the Celsius to Fahrenheit for the temperature. Then, assess the wind speed for surfing. Strong winds can affect wave size and safety. Since the sky is clear, there's good visibility. But with wind speed around 11.3 m/s, which is about 25 mph, that's quite strong. Surfers might need to consider if the wind is too strong for their skill level. Maybe mention that while the conditions are clear, the wind could impact the surfing experience. Suggest checking wave conditions or wind direction for better advice. Keep the answer friendly and helpful.
</think>

The current weather in San Diego is clear with a temperature of 70.5°F (

In [12]:
chat("What is the current weather in Los Angeles, and should I bring sunscreen?")

Tool: get_weather
<think>
Okay, the user is asking about the current weather in Los Angeles and whether they should bring sunscreen. Let me check the tool response. The temperature is 21.8°C, which is about 71.2°F. The wind speed is 16.2 km/h, and the condition is clear sky with a sun icon. Since the sky is clear and the temperature is mild, it's likely sunny. Sunscreen is recommended when the sun is strong, so even though it's not extremely hot, if it's sunny, sunscreen would be a good idea. I should mention the temperature in both Celsius and Fahrenheit, note the clear sky, and advise on sunscreen.
</think>

The current weather in Los Angeles is 21.8°C (71.2°F) with a clear sky and wind speed of 16.2 km/h. Since the sky is clear, it’s likely sunny, so bringing sunscreen would be a good idea to protect your skin from the sun. ☀️


- Ask about the company that we had put in the SQL database

In [13]:
chat("What roles are there in the company?")

Tool: execute_sql_query
<think>
Okay, the user asked, "What roles are there in the company?" and the tool response from execute_sql_query gave the results: ['Engineer', 'Manager', 'Analyst', 'HR']. I need to present these roles in a clear and natural way.

First, I should list each role, making sure they're all included. The user might be looking for a straightforward answer, so maybe just stating them out. But perhaps adding a sentence to explain that these are the roles available. Let me check if there's any need to mention the SQL query part, but since the user didn't ask about the query, just the roles, I can focus on the results. Also, the roles are in a list, so using bullet points or commas would be good. Wait, the example in the problem uses a list with each role on a separate line. So maybe I should format it similarly. Let me make sure there's no markdown, but the user said to reply as an answerer, so probably just plain text. Alright, I'll list each role separated by commas 

In [14]:
chat("Which employees are Engineers?")

Tool: execute_sql_query
<think>
Okay, the user asked, "Which employees are Engineers?" and the tool output is [{'results': [['Alice', 'Engineer']]}. So, I need to present this information clearly.

First, I should check if the query was executed correctly. The result shows Alice with the job title Engineer. The user probably wants to know the names of employees who are Engineers. Since there's only one result, I should mention Alice specifically. 

I should make sure to format the answer in a natural way, not just listing the data. Maybe start by stating that the query returned the result, then list the employee. Also, check if the user needs more details, but since the question is straightforward, just providing the answer should be enough. No need to convert units here because it's about job titles. Alright, the response should be clear and concise.
</think>

The query returned the following result:  
**Alice** is an Engineer.


- Run a prompt that is unrelated to any of the tools, and see what the LLM does.

In [15]:
chat("Foo Bar Foo Bar")

<think>
Okay, the user just said "Foo Bar Foo Bar". Hmm, that's interesting. Let me see what they need. The tools available are get_weather, get_current_time, and execute_sql_query. But the user's message doesn't mention anything related to weather, time, or a database query. They're just repeating "Foo Bar" twice. Maybe they're testing the system or trying to see if I can recognize a pattern. Since there's no explicit request for any of the tools, I should probably respond with a general message. Let me check if there's any hidden intent here. Maybe they want to know the time or weather for a specific city, but they didn't specify. Alternatively, they might be referring to a database table, but without more context, it's hard to tell. Since none of the tools are directly applicable, I'll just acknowledge their message and ask if they need anything else.
</think>

The message "Foo Bar Foo Bar" does not contain explicit requests for weather, time, or database queries. However, if you ha